In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyze a codebase with Gemini 1.5 Pro in Vertex AI 


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcode%2Fanalyze_codebase_with_gemini_1_5_pro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Eric Dong](https://github.com/gericdong), [Aakash Gouda](https://github.com/aksstar)|

## Overview

Gemini 1.5 Pro introduces a breakthrough long context window of up to 1 million tokens that can help seamlessly analyze, classify and summarize large amounts of content within a given prompt. With its long-context reasoning, Gemini 1.5 Pro can analyze an entire codebase for deeper insights.

In this tutorial, you learn how to analyze an entire codebase with Gemini 1.5 Pro and prompt the model to:

- **Analyze**: Summarize codebases effortlessly.
- **Guide**: Generate clear developer getting-started documentation.
- **Debug**: Uncover critical bugs and provide fixes.
- **Enhance**: Implement new features and improve reliability and security.


## Getting Started

### Install Vertex AI SDK for Python


In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                        gitpython \
                                        magika

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart runtime (Colab only)

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [3]:
import sys
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "qwiklabs-gcp-01-92978dcc88b4"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Please like share & subscribe to Techcps
# YouTube https://www.youtube.com/@techcps

print("Please like share & subscribe to Techcps https://www.youtube.com/@techcps")

### Import libraries

In [3]:
from IPython.core.interactiveshell import InteractiveShell
import IPython.display

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
)

In [4]:
# Define a helper function to call gemini and avoid rate limiting
import time

def call_gemini(prompt):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, stream=True)
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Cloning a codebase

You will use repo [Online Boutique](https://github.com/GoogleCloudPlatform/microservices-demo) as an example in this notebook. Online Boutique is a cloud-first microservices demo application. The application is a web-based e-commerce app where users can browse items, add them to the cart, and purchase them. This application consists of 11 microservices across multiple languages.

In [5]:
# The GitHub repository URL
repo_url = "https://github.com/GoogleCloudPlatform/microservices-demo"  # @param {type:"string"}

# The location to clone the repo
repo_dir = "./repo"

#### Define helper functions for processing GitHub repository

In [6]:
import os
from pathlib import Path
import shutil

import git
import magika
import requests

m = magika.Magika()


def clone_repo(repo_url, repo_dir):
    """Clone a GitHub repository."""

    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    os.makedirs(repo_dir)
    git.Repo.clone_from(repo_url, repo_dir)


def extract_code(repo_dir):
    """Create an index, extract content of code/text files."""

    code_index = []
    code_text = ""
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, repo_dir)
            code_index.append(relative_path)

            file_type = m.identify_path(Path(file_path))
            if file_type.output.group in ("text", "code"):
                try:
                    with open(file_path) as f:
                        code_text += f"----- File: {relative_path} -----\n"
                        code_text += f.read()
                        code_text += "\n-------------------------\n"
                except Exception:
                    pass

    return code_index, code_text


def get_github_issue(owner: str, repo: str, issue_number: str) -> str:
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
    }  # Set headers for GitHub API

    # Construct API URL
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"

    try:
        response_git = requests.get(url, headers=headers)
        response_git.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as error:
        print(f"Error fetching issue: {error}")  # Handle potential errors

    issue_data = response_git.json()
    if issue_data:
        return issue_data["body"]
    return ""

#### Create an index and extract content of a codebase

Clone the repo and create an index and extract content of code/text files.

In [7]:
clone_repo(repo_url, repo_dir)

code_index, code_text = extract_code(repo_dir)

## Analyzing the codebase with Gemini 1.5 Pro

With its long-context reasoning, Gemini 1.5 Pro can process the codebase and answer questions about the codebase.

#### Load the Gemini 1.5 Pro model

Learn more about the [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [8]:
from vertexai.generative_models import HarmCategory, HarmBlockThreshold

MODEL_ID = "gemini-1.5-pro"  # @param {type:"string"}

safety_config = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "You are a coding expert.",
        "Your mission is to answer all code related questions with given context and instructions.",
    ],
    safety_settings=safety_config
)

#### Define a helper function to generate a prompt to a code related question

In [9]:
def get_code_prompt(question):
    """Generates a prompt to a code related question."""

    prompt = f"""
    Questions: {question}

    Context:
    - The entire codebase is provided below.
    - Here is an index of all of the files in the codebase:
      \n\n{code_index}\n\n.
    - Then each of the files is concatenated together. You will find all of the code you need:
      \n\n{code_text}\n\n

    Answer:
  """

    return prompt

### 1. Summarizing the codebase


Generate a summary of the codebase.

In [10]:
question = """
  Give me a summary of this codebase, and tell me the top 3 things that I can learn from it.
"""

prompt = get_code_prompt(question)
contents = [prompt]

# Generate text using non-streaming method
responses = call_gemini(contents)

for response in responses:
  # Print generated text and usage metadata
  print(f"\nAnswer:\n{response.text}")
  print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')
  print(f"\nFinish reason:\n{response.candidates[0].finish_reason}")
  print(f"\nSafety settings:\n{response.candidates[0].safety_ratings}")


Answer:
```

Usage metadata:
{}

Finish reason:
0

Safety settings:
[]

Answer:

This codebase is a demo microservices application called "Online Boutique" developed

Usage metadata:
None

Finish reason:
0

Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.048828125
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.11572265625
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.12158203125
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.08740234375
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.11279296875
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.06640625
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.099609375
severity: HARM_SEVERITY_LOW
severity_score: 0.203125
]

Answer:
 by Google. It simulates an e-commerce website where users can browse products,

Usage metadata:
None

Finish reason:
0

Safety setti

### 2. Creating a developer getting started guide

Generate a getting started guide for developers. This sample uses the streaming option to generate the content.

In [11]:
question = """
  Provide a getting started guide to onboard new developers to the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```


## Onboarding Guide for New Developers to the Online Boutique Codebase

Welcome

 to the Online Boutique microservices demo application! This guide will help you get started

 with the codebase.

**1. Understand the Architecture**

Online Boutique is a cloud-native microservices demo application, demonstrating how to modernize enterprise applications

 using Google Cloud and other cloud-agnostic tools. It consists of 11 microservices written in various languages, communicating over gRPC.

- **

...

 branding changes.
- **Terraform:** The `./terraform` directory has Terraform scripts for provisioning a GKE cluster and deploying the application.

**4. Local Development**

You can develop and test individual services locally using Docker. The `./skaffold.yaml` file helps to streamline local development with

 Skaffold, a tool for building and deploying applications.

- **Development Guide:** Refer to `./docs/development-guide.md` for detailed instructions on setting up your local environment.

**5. Testing**

Unit tests are provided for each microservice. You can run these tests locally before submitting any

 changes.

- **Test Scenarios:**  Explore the `./kustomize/tests` directory for examples of different Kustomize configurations used for testing.

**6. Contribute**

- **Contribution Guide:** Before contributing, read the `./.github/CONTRIBUTING.md` file for guidelines and the

 contribution process.
- **Pull Requests:** Submit your changes as pull requests to the main branch.

**7. Resources**

- **README:** The root `README.md` file is your starting point for understanding Online Boutique.
- **Documentation:**  The `./docs` directory contains additional documentation,

 including development guides, deployment options, and release instructions.
- **GitHub Issues:** Use GitHub issues for bug reports, feature requests, or questions.

We encourage you to dive into the code, experiment with different deployment options, and contribute to making Online Boutique even better!
```

### 3. Finding bugs

Find the top 3 most severe issues in the codebase.

In [12]:
question = """
  Find the top 3 most severe issues in the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
    Questions: 
  Find the top 3 most severe issues

 in the codebase.


    Context:
    - The entire codebase

 is provided below.
    - Here is an index of all of the files in the codebase:
      

['README.md', '.gitattributes

', '.gitignore', 'skaffold.yaml', 'cloudbuild.yaml', '.editorconfig', 'LICENSE', 'docs/product-requirements.md

...

mesh/docs/strengthen-app-security)
- [From edge to mesh: Exposing service mesh applications through GKE Ingress](https://cloud.google.com/architecture/exposing-service-mesh-apps-through-gke-ingress)
- [Take the first step toward S

RE with Cloud Operations Sandbox](https://cloud.google.com/blog/products/operations/on-the-road-to-sre-with-cloud-operations-sandbox)
- [Deploying the Online Boutique sample application on Cloud Service Mesh](https://cloud.google.com/service-mesh

/docs/onlineboutique-install-kpt)
- [Anthos Service Mesh Workshop: Lab Guide](https://codelabs.developers.google.com/codelabs/anthos-service-mesh-workshop)
- [KubeCon EU 2019 - Reinventing Networking: A

 Deep Dive into Istio's Multicluster Gateways - Steve Dake, Independent](https://youtu.be/-t2BfT59zJA?t=982)
- Google Cloud Next'18 SF
  - [Day 1 Keynote](https://youtu.be/v

J9OaAqfxo4?t=2416) showing GKE On-Prem
  - [Day 3 Keynote](https://youtu.be/JQPOPV_VH5w?t=815) showing Stackdriver
    APM (Tracing, Code Search,

### 4. Fixing bug

Find the most severe issue in the codebase that can be fixed and provide a code fix for it.


In [13]:
question = """
  Find the most severe bug in the codebase that you can provide a code fix for.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```

diff
--- a/src/productcatalogservice/catalog_loader.go


+++ b/src/productcatalogservice/product_catalog.go


@@ -162,7 +162,7 @@
 }
 
 func (p *productCatalog) ListProducts(context.Context

, *pb.Empty) (*pb.ListProductsResponse, error) {
-	time.Sleep(extraLatency)
+	p.apply

ExtraLatency()
 
 	return &pb.ListProductsResponse{Products: p.parseCatalog()}, nil
 }
@@ -184,7 +184,7 @@
 }
 
 func (

p *productCatalog) SearchProducts(ctx context.Context, req *pb.SearchProductsRequest) (*pb.SearchProductsResponse, error) {
-	time.Sleep(extraLatency)
+	p.applyExtra

Latency()
 
 	var ps []*pb.Product
 	for _, product := range p.parseCatalog() {
@@ -205,6 +205,10 @@
 	}
 
 	return p.catalog.Products
+}
+
+

func (p *productCatalog) applyExtraLatency() {
+	time.Sleep(extraLatency)
 }
 
 -------------------------

```

### 5. Implementing a feature request using Function Calling

Generate code to implement a feature request.

Get feature request text from GitHub Issue

In [14]:
# Function declaration with detailed docstring
extract_details_from_url_func = FunctionDeclaration(
    name="extract_details_from_url",
    description="Extracts owner, repository name, and issue number details from a GitHub issue URL",
    parameters={
        "type": "object",
        "properties": {
            "owner": {
                "type": "string",
                "description": "The owner of the GitHub repository.",
            },
            "repo": {
                "type": "string",
                "description": "The name of the GitHub repository.",
            },
            "issue_number": {
                "type": "string",
                "description": "The issue number to fetch the body of.",
            },
        },
    },
)

# Tool definition
extraction_tool = Tool(function_declarations=[extract_details_from_url_func])

FEATURE_REQUEST_URL = (
    "https://github.com/GoogleCloudPlatform/microservices-demo/issues/2205"
)

# Prompt content
prompt_content = f"What is the feature request of the following {FEATURE_REQUEST_URL}"

# Model generation with tool usage
response = model.generate_content(
    [prompt_content],
    generation_config=GenerationConfig(temperature=0),
    tools=[extraction_tool],
)
# Extract parameters from model response
function_call = response.candidates[0].function_calls[0]

# Fetch issue details from GitHub API if function call matches
if function_call.name == "extract_details_from_url":
    issue_body = get_github_issue(
        function_call.args["owner"],
        function_call.args["repo"],
        function_call.args["issue_number"],
    )

IPython.display.Markdown(f"Feature Request:\n{issue_body}")

Feature Request:
### Describe request or inquiry 
helm chart frontend-external support config service type nodeport, like this
```
helm install xxx --set frontend.service.type=NodePort
```

### What purpose/environment will this feature serve? 

This feature enables quick access to the front-end web interface of microservices without the need for additional configuration work.

I hope to quickly access the web interface after deploying this microservice in the local environment, without the need for additional loadbalancer or ingress configuration, just nodeport is enouth.

But now that the deployment is complete, I must manually edit the service and modify the nodeport. If Helm provides parameters to set the nodeport, I don't need to。


Use the GitHub Issue text to implement the feature request

In [15]:
# Combine feature request with URL and get code prompt
question = (
    "Implement the following feature request" + FEATURE_REQUEST_URL + "\n" + issue_body
)

prompt = get_code_prompt(question)

# Generate code response
response = call_gemini([prompt])

for r in response:
    IPython.display.Markdown(r.text)  # Display in Markdown format

```

diff
--- a/helm-chart/templates/frontend.yaml
+++

 b/helm-chart/templates/frontend.yaml
@@ -72

8,6 +728,8 @@
     - "*"
     gateway:
       name: asm-ingressgateway
+      # The default

 Anthos Service Mesh namespace.  Can be overriden.
       namespace: asm-ingress
       labelKey: asm
       labelValue: ingress

gateway
@@ -749,6 +751,8 @@
     app: {{ .Values.frontend.name }}
 spec:
   type: ClusterIP
+  # Anthos Service Mesh requires the frontend

 to be headless.
+  clusterIP: None
   selector:
     app: {{ .Values.frontend.name }}
   ports:
@@ -762,6 +766,8 @@
   

name: {{ .Values.frontend.name }}-external
   namespace: {{ .Release.Namespace }}
 spec:
+  # When set to NodePort, you can access the frontend via the node's IP and port.
+  type: {{ .Values.frontend.service.type }}


   type: LoadBalancer
   selector:
     app: {{ .Values.frontend.name }}

```

### 6. Creating a troubleshooting guide

Create a troubleshooting guide to help resolve common issues.

In [16]:
question = """
    Provide a troubleshooting guide to help resolve common issues.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```


## Online Boutique Troubleshooting Guide

This guide helps you troubleshoot common issues encountered when

 running the Online Boutique microservices demo application.

**General Troubleshooting Tips**



* **Check the logs:** Each service has its logging mechanism. Consult the service's README for instructions on accessing its logs. Kubernetes deployments typically log to stdout

/stderr, which can be viewed using `kubectl logs <pod-name>`.
* **Monitor resource usage:** Monitor the CPU and memory consumption of your

...

Python):**
    * **Email Sending:** The actual email sending functionality is not implemented. The service logs the email details, but no email is sent.
* **shippingservice (Go):**
    * **Quote Generation:** The service generates a fixed quote based on the number of items, not the

 actual products.
* **loadgenerator (Python):**
    * **Frontend Connection:** Ensure the `FRONTEND_ADDR` environment variable is set to the correct address of the frontend service.
* **currencyservice (Node.js):**
    * **Currency Data:** The service uses a

 predefined set of currency conversion rates. Ensure the `currency_conversion.json` file is correctly loaded.

**Getting Help**

If you encounter an issue not covered in this guide or need further assistance, please [create a GitHub issue](https://github.com/GoogleCloudPlatform/microservices-demo/

issues/new/choose) providing details about the problem, your deployment environment, and any relevant logs or error messages.
```

This troubleshooting guide provides a starting point for addressing common problems. By following the tips and checking for service-specific issues, you can efficiently diagnose and resolve issues you encounter while running the Online

 Boutique application. 


### 7. Making the app more reliable

Recommend best practices to make the application more reliable.


In [17]:
question = """
  How can I make this application more reliable? Consider best practices from https://www.r9y.dev/
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import os
import random
import time
import traceback
from concurrent

 import futures

import googlecloudprofiler
from google.auth.exceptions import Default

CredentialsError
import grpc

import demo_pb2
import demo_pb2_grpc
from grpc_health.v1 import health_pb2


from grpc_health.v1 import health_pb2_grpc

from opentelemetry import trace
from opentelemetry.instrumentation

...

Stub(channel)
            # Test connection
            product_catalog_stub.ListProducts(demo_pb2.Empty())
            logger.info("Successfully connected to product catalog service.")
            break
        except grpc.RpcError as e:
            logger.warning(f"Failed to connect

 to product catalog service. Retry {retry}: {e.details()}")
            time.sleep(retry * 2)  # Exponential backoff
    else:
        logger.fatal("Could not connect to product catalog service after retries. Exiting.")
        exit(1)

    # create

 gRPC server
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))

    # add class to gRPC server
    service = RecommendationService()
    demo_pb2_grpc.add_RecommendationServiceServicer_to_server(service, server)


    health_pb2_grpc.add_HealthServicer_to_server(service, server)

    # start server
    logger.info("listening on port: " + port)
    server.add_insecure_port('[::]:'+port)
    server.start()



    # keep alive
    try:
         while True:
            time.sleep(10000)
    except KeyboardInterrupt:
            server.stop(0)
```

### 8. Making the app more secure

Recommend best practices to make the application more secure.

In [18]:
question = """
  How can you secure the application?
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

This

 application is the "Online Boutique" microservices demo. It is a learning tool

 and not designed for production environments. Security measures mentioned in the documentation are for demonstration

 purposes and should not be considered production-ready. 

Here's a breakdown of security considerations based on the provided code and context:

**General Security

 Considerations:**

* **Not Production-Ready:** This application is expressly not intended for production use. The documentation emphasizes its role as a demo and learning tool for

...

 Istio to manage traffic, add observability, and enhance security. It mentions mTLS, JWT, and Authorization features but does not implement them in detail.
* **Workload Identity:** This feature, highlighted for integration with Cloud Trace in `kustomize/components/google-cloud-operations`, allows Kubernetes Service

 Accounts to impersonate Google Service Accounts. Correct configuration is crucial to limit permissions and prevent privilege escalation.

**Recommendations for Hypothetical Productionization:**

If you were to hypothetically use this as a base for a production application, you would need to implement significant security hardening:

* **Authentication and Authorization:** Implement a

 robust system for user authentication and fine-grained authorization to control access to resources and operations.
* **Secure External Service Integrations:** Replace mock services with real integrations and implement secure communication channels (e.g., HTTPS, API keys, service accounts with least privilege).
* **Input Validation:**  Strengthen input validation across

 all services to prevent injection attacks and data corruption.
* **Secrets Management:** Store sensitive data (e.g., database credentials, API keys) securely using a secrets management system like Google Cloud Secret Manager.
* **Regular Security Audits:** Conduct regular security audits and penetration testing to identify and mitigate vulnerabilities.

**

Conclusion:**

The Online Boutique demo provides valuable insights into microservices architecture and related technologies. However, it's essential to understand its limitations as a non-production system.  Treat the security features as learning examples and **do not deploy this application in a production environment without thorough security hardening.** 


### 9. Learning the codebase

Create a quiz about the concepts used in the codebase.

In [19]:
question = """
  Create a quiz about the concepts used in my codebase to help me solidify my understanding.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import os
import random

class Question:
    def __init

__(self, question_text, answer_choices, correct_answer_index

):
        self.question_text = question_text
        self.answer_choices = answer_choices
        self.correct_answer_index

 = correct_answer_index

    def check_answer(self, user_answer_index):
        return user_answer_index == self.

...

", "SIGINT", "SIGKILL", "SIGUSR1"],
        3
    ),
    Question(
        "Which deployment variation changes all branding to Google Cloud's fictitious company?",
        ["Memorystore", "Google Cloud Operations", "Spanner", "Cymbal Shops Branding"],
        3


    )
]

def run_quiz(questions):
    score = 0
    random.shuffle(questions)
    
    os.system('cls' if os.name == 'nt' else 'clear')

    for question in questions:
        question.display_question()


        
        while True:
            try:
                answer = int(input("Enter the number of your answer: ")) - 1
                if 0 <= answer < len(question.answer_choices):
                    break
                else:
                    print("Invalid input. Please enter a number

 from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")
        
        if question.check_answer(answer):
            print("Correct!")
            score += 1
        else:
            print(f"Incorrect. The correct answer is {question

.answer_choices[question.correct_answer_index]}.")
        print()

    print(f"You got {score} out of {len(questions)} questions correct.")

if __name__ == "__main__":
    run_quiz(questions)
```

### 10. Creating a quickstart tutorial

Create an end-to-end quickstart tutorial for a specific component.


In [20]:
question = """
  Please write an end-to-end quickstart tutorial that introduces AlloyDB,
  shows how to configure it with the CartService,
  and highlights key capabilities of AlloyDB in context of the Online Boutique application.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = call_gemini(contents)
for response in responses:
    IPython.display.Markdown(response.text)

```

alloydb
## Quickstart: Integrate AlloyDB with the CartService in Online

 Boutique

This tutorial demonstrates how to configure the CartService of the Online Boutique micros

ervices demo application to use AlloyDB as its persistent data store. We'll guide you through provisioning an AlloyDB database, configuring the CartService to connect,

 and highlight AlloyDB's advantages in this scenario.

**Prerequisites:**

* A Google Cloud project with billing enabled.
* Basic familiarity with Kubernetes and

...

)
               - name: ALLOYDB_PASSWORD
                 value: <your_alloydb_password> 
               - name: ALLOYDB_DATABASE
                 value: carts
     ```

3. **Deploy and Test:**

   * Deploy your updated `cartservice` deployment to your Kubernetes

 cluster.
   * Access the Online Boutique frontend and add items to the cart.
   * Verify the CartService is correctly storing and retrieving cart data from AlloyDB.

**Key Capabilities of AlloyDB:**

* **High Performance:** AlloyDB delivers superior performance compared to standard PostgreSQL, making it ideal for transaction

-intensive applications like the CartService.
* **Scalability:** AlloyDB's read pool instances provide horizontal scalability, allowing you to handle high volumes of read traffic from the frontend.
* **Managed Service:** AlloyDB is a fully managed service, eliminating the need for database administration tasks. This allows you to

 focus on application development rather than database management.
* **Integration with Google Cloud:** AlloyDB integrates seamlessly with other Google Cloud services, such as Secret Manager for secure password management.

By following these steps, you've successfully integrated AlloyDB with the CartService of Online Boutique, leveraging its high performance and scalability to

 enhance the application's responsiveness and resilience. 
```

### 11. Creating a Git Changelog Generator

Understanding changes made between Git commits and highlighting the most important aspects of the changes.

In [21]:
### Fetches commit IDs from a local Git repository on a specified branch.

repo = git.Repo(repo_dir)
branch_name = "main"
commit_ids = [
    commit.hexsha for commit in repo.iter_commits(branch_name)
]  # A list of commit IDs (SHA-1 hashes) in reverse chronological order (newest first)

if len(commit_ids) >= 2:
    diff_text = repo.git.diff(commit_ids[0], commit_ids[1])

    question = """
      Given the above git diff output, Summarize the important changes made.
    """

    prompt = diff_text + question + code_text
    contents = [prompt]

    responses = call_gemini(contents)
    for response in responses:
        IPython.display.Markdown(response.text)

The

 update introduces several changes, including:

- **Configuration for end-of-

line (EOL) handling:** A `.gitattributes` file is added to convert

 line endings to LF for all text files, ensuring consistent handling across operating systems.
- **Enhanced `.gitignore` file:** The `.gitignore` file

 is expanded to exclude additional files and directories, such as `.kubernetes-manifests-*`, `.terraform.lock.hcl`, `.terraform/*`, `Docker

...

 including `.deploystack/test`, `.deploystack/test.yaml`, `.deploystack/deploystack.yaml`, `.deploystack/messages/description.txt`, `.deploystack/messages/success.txt`, and `.deploystack/scripts/preinit.sh`. These changes aim to improve the testing process

 for DeployStack deployments.
- **Enhancements to Helm chart:**  The `helm-chart/` directory sees updates to its `README.md`, `Chart.yaml`, and `values.yaml` files. These changes provide instructions for deploying Online Boutique using the Helm chart, detail configuration options, and specify

 version information.
- **Addition of shopping assistant service:** Files related to a new `shoppingassistantservice`, including Dockerfile, `shoppingassistantservice.py`, `requirements.txt`, `requirements.in`, `kustomization.yaml`, and shell scripts for deployment and database management, are added.

 This service likely extends the functionality of Online Boutique, potentially integrating AI features or enhancing the shopping experience.

Overall, the changes reflect improvements in various areas of the project, including code style consistency, build and deployment processes, documentation, security, and support for new tools and services. The addition of a shopping assistant service suggests

 a significant functional enhancement to the Online Boutique application.


## Conclusion

In this tutorial, you've learned how to use the Gemini 1.5 Pro to analyze a codebase and prompt the model to:

- Summarize codebases effortlessly.
- Generate clear developer getting-started documentation.
- Uncover critical bugs and provide fixes.
- Implement new features and improve reliability and security.
- Understanding changes made between Git commits